In [ ]:
try:
    import mlxtend

    # Si mlxtend est déjà installé
    print("MLxtend est déjà installé (version {})".format(mlxtend.__version__))

except ImportError:
    # Si mlxtend n'est pas installé, installez-le
    !pip install mlxtend
    print("MLxtend a été installé avec succès.")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import itertools

## Partie 1

Sélection des applications téléchargées plus de 1M de fois avec un score supérieur à 4.

But : essayer de trouver des patterns revenant souvent. Par exemple, des couples genre principal - genre secondaire.

### Création du dataframe

In [ ]:
# Lecture du dataset
df = pd.read_csv("data/clean_dataset_googleplaystore_cat_str.csv")
df.head(2)

In [ ]:
def create_a_priori_df(df, installs = 1000000, rating=0, reviews=0, genre_secondaire=False, price=True, category=False):
    ''' 
        Crée le data frame utilisé pour l'apriori.
        
        @param df : dataframe, dataframe à utiliser pour créer le dataframe pour l'apriori.
        @param installs : int, nombre d'installations minimum pour qu'une application soit considérée comme populaire.
        @param rating : float, note minimum pour qu'une application soit considérée comme populaire.
        @param reviews : int, nombre de reviews minimum pour qu'une application soit considérée comme populaire.
        @param genre_secondaire : bool, si True, les genres secondaires sont pris en compte, sinon, ils sont ignorés.
        @param price : bool, si True, le prix des applications est prise en compte, sinon, il esy ignoré.
        
        @return : dataframe, dataframe pour l'apriori.
    '''
    df_apriori = df.copy()
    
    # Ajoute les colonnes genre principal et genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre   
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
    if genre_secondaire:
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)
    if category:
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["Category"])], axis=1)
    
    # Sélection des colonnes nous intéressant
    selected_columns = ["App", "Price", "Rating", "Reviews", "GenrePrincipal", "GenreSecondaire", "Category", "Installs"]
    df_apriori = df_apriori[selected_columns]
    
    # Prix = 1 si l'application est payante, 0 sinon.
    if price:
        df_apriori["Price"] = df_apriori["Price"].apply(lambda x: 1 if x == 0 else 0)
    else:
        df_apriori = df_apriori.drop(columns=["Price"])
    
    # Si rating != 0, Rating = 1 si l'application a une note supérieure à rating, 0 sinon.
    # Sinon suppression de la colonne Rating.
    if rating != 0:
        df_apriori["Rating"] = df_apriori["Rating"].apply(lambda x: 1 if x > rating else 0)
    else:
        df_apriori = df_apriori.drop(columns=["Rating"])
    
    # Si reviews != 0, Reviews = 1 si l'application a plus de reviews que reviews, 0 sinon.
    # Sinon suppression de la colonne Reviews.
    if reviews != 0:
        df_apriori["Reviews"] = df_apriori["Reviews"].apply(lambda x: 1 if x > reviews else 0)
    else:
        df_apriori = df_apriori.drop(columns=["Reviews"])
        
    df_apriori["Installs"] = df_apriori["Installs"].apply(lambda x: 1 if x > installs else 0)

    # Sépare les colonnes genre principal et genre secondaire en colonnes correspondant à chaque genre. Concaténation
    # au nom de ces colonnes de 1 si le genre est le genre principal, 2 si le genre est le genre secondaire. 
    df_apriori["GenrePrincipal"] = df_apriori["GenrePrincipal"].astype(str) + "1"
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
    # Si genre_secondaire = True, ajoute les colonnes genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre.
    # Sinon, supprime la colonne GenreSecondaire.
    if genre_secondaire:
        df_apriori["GenreSecondaire"] = df_apriori["GenreSecondaire"].astype(str) + "2"
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)    
        #Renommage de la colonne "nan2" en "NoSecondaryGenre"
        df_apriori = df_apriori.rename(columns={"nan2": "NoSecondaryGenre"})
    else:
        df_apriori = df_apriori.drop(columns=["GenreSecondaire"])
    
    if category:
        df_apriori["Category"] = df_apriori["Category"].astype(str)
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["Category"])], axis=1)
    else:
        df_apriori = df_apriori.drop(columns=["Category"])

    # Enlever les lignes où installs n'est pas à 1
    df_apriori = df_apriori[df_apriori["Installs"] == 1]

    return df_apriori

In [ ]:
def pivot(df_apriori, reviews=0):
    '''
        Pivote le dataframe df :
            - une ligne par application
            - une colonne pour le prix
            - une colonne pour le nombre de reviews, si reviews != 0
            - une colonne par genre
    '''
    #Exclure des colonnes
    excluded_columns = ["App", "Installs",  "GenrePrincipal", "GenreSecondaire", "Category", "Rating"]
    
    
    pivoted = df_apriori.pivot_table(index='App', columns=None, values=[col for col in df_apriori.columns if col not in excluded_columns], aggfunc='mean')


    # Afficher la liste des colonnes sélectionnées
    pivoted = pivoted.fillna(0)
    pivoted = pivoted.astype(int)
    
    pivoted.to_csv("data/apriori.csv")
    
    return pivoted
    

In [ ]:
def support_options(df, installs_threshold=1000000, reviews_threshold=0, ratings_threshold=0, genre_secondaire=False, price=True, category=False):
    df_apriori = create_a_priori_df(df, installs=installs_threshold, rating=ratings_threshold, reviews=reviews_threshold, genre_secondaire=genre_secondaire, price=price, category=category)
    pivoted = pivot(df_apriori, reviews=reviews_threshold)
    supports = apriori(pivoted, min_support=0.005,max_len=3, use_colnames=True)
    
    return supports

### Supports calculés pour différentes valeurs d'installations, reviews, ratings et genre secondaire

In [ ]:
installs_values = [1000000, 5000000, 10000000, 50000000, 100000000]
reviews_values = [0, 100, 500, 1000, 1500, 30000]
ratings_values = [0, 4, 4.5]
genre_sec_values = [True, False]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, ratings_values, reviews_values, genre_sec_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_categories.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, reviews_threshold, rating_threshold, genre_sec_value = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options(df, installs_threshold=installs_threshold, reviews_threshold=reviews_threshold, 
                                 ratings_threshold=rating_threshold, genre_secondaire=genre_sec_value, price=True, category=True)

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Reviews>={reviews_threshold}, Ratings>={rating_threshold}, GenreSec={genre_sec_value}:\n")
        output_file.write(str(result.sort_values(by="support", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")

### Supports calculés pour les genres principaux et genre secondaires (Variation des Installs, Reviews, et Ratings)

In [ ]:
installs_values = [1000000, 5000000, 10000000, 50000000, 100000000]
ratings_values = [0, 4, 4.5]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, ratings_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_no_2nd_genre_category.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, rating_threshold = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options(df, installs_threshold=installs_threshold, reviews_threshold=0, 
                                 ratings_threshold=rating_threshold, genre_secondaire=False, price=False, category=True)

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Reviews>={reviews_threshold}, Ratings>={rating_threshold}:\n")
        output_file.write(str(result.sort_values(by="support", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")

### Supports calculés pour les couples genres catégorie

In [ ]:
installs_values = [1000000, 5000000, 10000000, 50000000, 100000000]
ratings_values = [0, 4, 4.5]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, ratings_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_couples_genre_categorie.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, rating_threshold = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options(df, installs_threshold=installs_threshold, reviews_threshold=0, 
                                 ratings_threshold=rating_threshold, genre_secondaire=False, price=False, category=True)
        result["len"] = result["itemsets"].apply(lambda x: len(x))

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Ratings>={rating_threshold}:\n")
        output_file.write(str(result[result["len"]==2].sort_values(by="support", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")

### Support rules

In [ ]:
installs_values = [1000000, 5000000, 10000000, 50000000, 100000000]
ratings_values = [0, 4, 4.5]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, ratings_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_couples_genres_rules.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, rating_threshold = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options(df, installs_threshold=installs_threshold, reviews_threshold=0, 
                                 ratings_threshold=rating_threshold, genre_secondaire=True, price=False)
        rules = association_rules(result, min_threshold=0.5)

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Ratings>={rating_threshold}:\n")
        output_file.write(str(rules.sort_values(by="lift", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")

## Partie 2

Dans cette partie, plutôt que d'inférer sur aucune colonne, nous avons passé les colonnes Installs et Ratings, pour des valeurs respectivement supérieures à 1 million et 4.

In [ ]:
def create_a_priori_df_col_installs(df, installs = 1000000, rating=4, reviews=0, genre_secondaire=False, price=True):
    ''' 
        Crée le data frame utilisé pour l'apriori.
        
        @param df : dataframe, dataframe à utiliser pour créer le dataframe pour l'apriori.
        @param installs : int, nombre d'installations minimum pour qu'une application soit considérée comme populaire.
        @param rating : float, note minimum pour qu'une application soit considérée comme populaire.
        @param reviews : int, nombre de reviews minimum pour qu'une application soit considérée comme populaire.
        @param genre_secondaire : bool, si True, les genres secondaires sont pris en compte, sinon, ils sont ignorés.
        @param price : bool, si True, le prix des applications est prise en compte, sinon, il esy ignoré.
        
        @return : dataframe, dataframe pour l'apriori.
    '''
    df_apriori = df.copy()
    
    # Ajoute les colonnes genre principal et genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre   
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
    if genre_secondaire:
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)

    
    # Sélection des colonnes nous intéressant
    selected_columns = ["App", "Price", "Rating", "Reviews", "GenrePrincipal", "GenreSecondaire", "Installs"]
    df_apriori = df_apriori[selected_columns]
    
    # Prix = 1 si l'application est payante, 0 sinon.
    if price:
        df_apriori["Price"] = df_apriori["Price"].apply(lambda x: 1 if x == 0 else 0)
    else:
        df_apriori = df_apriori.drop(columns=["Price"])
    
    # Si reviews != 0, Reviews = 1 si l'application a plus de reviews que reviews, 0 sinon.
    # Sinon suppression de la colonne Reviews.
    if reviews != 0:
        df_apriori["Reviews"] = df_apriori["Reviews"].apply(lambda x: 1 if x > reviews else 0)
    else:
        df_apriori = df_apriori.drop(columns=["Reviews"])

    # Sépare les colonnes genre principal et genre secondaire en colonnes correspondant à chaque genre. Concaténation
    # au nom de ces colonnes de 1 si le genre est le genre principal, 2 si le genre est le genre secondaire. 
    df_apriori["GenrePrincipal"] = df_apriori["GenrePrincipal"].astype(str) + "1"
    df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenrePrincipal"])], axis=1)
    # Si genre_secondaire = True, ajoute les colonnes genre secondaire avec comme valeur 1 pour l'indice du tableau qui est le bon genre.
    # Sinon, supprime la colonne GenreSecondaire.
    if genre_secondaire:
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)
        df_apriori["GenreSecondaire"] = df_apriori["GenreSecondaire"].astype(str) + "2"
        df_apriori = pd.concat([df_apriori, pd.get_dummies(df_apriori["GenreSecondaire"])], axis=1)    
        #Renommage de la colonne "nan2" en "NoSecondaryGenre"
        df_apriori = df_apriori.rename(columns={"nan2": "NoSecondaryGenre"})
    else:
        df_apriori = df_apriori.drop(columns=["GenreSecondaire"])

    # Enlever les lignes où installs est inférieur à 1000000
    df_apriori = df_apriori[df_apriori["Installs"] >= installs]
    df_apriori = df_apriori[df_apriori["Rating"] >= rating]
    
    return df_apriori

In [ ]:
def pivot_col_installs(df_apriori, reviews=0):
    '''
        Pivote le dataframe df :
            - une ligne par application
            - une colonne pour le prix
            - une colonne pour le nombre de reviews, si reviews != 0
            - une colonne par genre
    '''
    #Exclure des colonnes
    excluded_columns = ["App", "Installs",  "GenrePrincipal", "GenreSecondaire", "Rating"]
    
    
    pivoted = df_apriori.pivot_table(index='App', columns=["Installs", "Rating"], values=[col for col in df_apriori.columns if col not in excluded_columns], aggfunc='mean')


    # Afficher la liste des colonnes sélectionnées
    pivoted = pivoted.fillna(0)
    pivoted = pivoted.astype(int)
    
    pivoted.to_csv("data/apriori_columns_installs.csv")
    
    return pivoted

In [ ]:
def support_options_col_installs(df, installs_threshold=1000000, reviews_threshold=0, ratings_threshold=4, genre_secondaire=False, price=True):
    df_apriori = create_a_priori_df_col_installs(df, installs=installs_threshold, rating=ratings_threshold, reviews=reviews_threshold, genre_secondaire=genre_secondaire, price=price)
    pivoted = pivot_col_installs(df_apriori, reviews=reviews_threshold)
    supports = apriori(pivoted, min_support=0.005,max_len=3, use_colnames=True)
    
    return supports

### Price, Reviews, GenrePrincipal et GenreSecondaire

In [ ]:
installs_values = [1000000, 10000000, 100000000]
reviews_values = [0, 100, 500, 1000, 1500, 30000]
ratings_values = [4, 4.5]
genre_sec_values = [True, False]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, reviews_values, ratings_values, genre_sec_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_part2.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, reviews_threshold, rating_threshold, genre_sec_value = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options_col_installs(df, installs_threshold=installs_threshold, reviews_threshold=reviews_threshold, 
                                 ratings_threshold=rating_threshold, genre_secondaire=genre_sec_value, price=True)

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Reviews>={reviews_threshold}, Ratings>={rating_threshold}, GenreSec={genre_sec_value}:\n")
        output_file.write(str(result.sort_values(by="support", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")

### GenrePrincipal et GenreSecondaire

In [ ]:
installs_values = [1000000, 5000000, 10000000, 50000000, 100000000]
ratings_values = [0, 4, 4.5]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, ratings_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_no_price_part2.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, rating_threshold = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options_col_installs(df, installs_threshold=installs_threshold, reviews_threshold=0, 
                                 ratings_threshold=rating_threshold, genre_secondaire=True, price=False)

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Reviews>={reviews_threshold}, Ratings>={rating_threshold}:\n")
        output_file.write(str(result.sort_values(by="support", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")

### Couples (GenrePrincipal, GenreSecondaire)

In [ ]:
installs_values = [1000000, 5000000, 10000000, 50000000, 100000000]
ratings_values = [0, 4, 4.5]

# Générer toutes les combinaisons possibles
combinations = list(itertools.product(installs_values, ratings_values))

# Créer un fichier pour enregistrer les résultats
output_file_path = "./output/resultats_apriori_couples_genres_part2.txt"

with open(output_file_path, "w") as output_file:
    # Itérer sur chaque combinaison
    for i, combination in enumerate(combinations, start=1):
        installs_threshold, rating_threshold = combination

        # Appeler la fonction custom_apriori avec la combinaison actuelle
        result = support_options_col_installs(df, installs_threshold=installs_threshold, reviews_threshold=0, 
                                 ratings_threshold=rating_threshold, genre_secondaire=True, price=False)
        result["len"] = result["itemsets"].apply(lambda x: len(x))

        # Écrire les résultats dans le fichier
        output_file.write(f"Résultat {i} pour Installs>={installs_threshold}, Ratings>={rating_threshold}:\n")
        output_file.write(str(result[result["len"]==2].sort_values(by="support", ascending=False).head()) + "\n\n")

print(f"Les résultats ont été enregistrés dans {output_file_path}")